<a href="https://colab.research.google.com/github/Svanzi/Music-Recommender-System/blob/main/Music_Recommender_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/Svanzi/Music-Recommender-System

fatal: destination path 'Music-Recommender-System' already exists and is not an empty directory.


In [ ]:
import os, pandas as pd, numpy as np, scipy.sparse as sp

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
artists = '/content/drive/MyDrive/University/SII/hetrec2011-lastfm-2k/artists.dat'
user_artists = '/content/drive/MyDrive/University/SII/hetrec2011-lastfm-2k/user_artists.dat'
user_friends = '/content/drive/MyDrive/University/SII/hetrec2011-lastfm-2k/user_friends.dat'
user_taggedartists = '/content/drive/MyDrive/University/SII/hetrec2011-lastfm-2k/user_taggedartists.dat'
user_taggedartists_timestamps = '/content/drive/MyDrive/University/SII/hetrec2011-lastfm-2k/user_taggedartists-timestamps.dat'

df_artist = pd.read_csv(artists, sep="\t", header=0)
df_user_artists = pd.read_csv(user_artists, sep="\t", header=0)
df_user_friends = pd.read_csv(user_friends, sep="\t", header=0)
df_user_taggedartists = pd.read_csv(user_taggedartists, sep="\t", header=0)
df_user_taggedartists_timestamps = pd.read_csv(user_taggedartists_timestamps, sep="\t", header=0)

Rinominare le colonne con termini più autoesplicativi

In [ ]:
user_artists = df_user_artists.rename(columns={'userID': 'user_id', 'artistID': 'artist_id', 'weight': 'play_count'})
display(user_artists.head())
display(print("\n\n"))
display(user_artists.info())

,user_id,artist_id,play_count
0,2,51,13883
1,2,52,11690
2,2,53,11351
3,2,54,10300
4,2,55,8983


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92834 entries, 0 to 92833
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   user_id     92834 non-null  int64
 1   artist_id   92834 non-null  int64
 2   play_count  92834 non-null  int64
dtypes: int64(3)
memory usage: 2.1 MB


None

In [ ]:
artists = df_artist.rename(columns={'id': 'artist_id', 'name': 'artist_name'})
display(artists.head())
display(print("\n\n"))
display(artists.info())

,artist_id,artist_name,url,pictureURL
0,1,MALICE MIZER,http://www.last.fm/music/MALICE+MIZER,http://userserve-ak.last.fm/serve/252/10808.jpg
1,2,Diary of Dreams,http://www.last.fm/music/Diary+of+Dreams,http://userserve-ak.last.fm/serve/252/3052066.jpg
2,3,Carpathian Forest,http://www.last.fm/music/Carpathian+Forest,http://userserve-ak.last.fm/serve/252/40222717...
3,4,Moi dix Mois,http://www.last.fm/music/Moi+dix+Mois,http://userserve-ak.last.fm/serve/252/54697835...
4,5,Bella Morte,http://www.last.fm/music/Bella+Morte,http://userserve-ak.last.fm/serve/252/14789013...


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17632 entries, 0 to 17631
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   artist_id    17632 non-null  int64 
 1   artist_name  17632 non-null  object
 2   url          17632 non-null  object
 3   pictureURL   17188 non-null  object
dtypes: int64(1), object(3)
memory usage: 551.1+ KB


None

Rimozione eventuali duplicati

In [ ]:
user_artists = user_artists.drop_duplicates(["user_id","artist_id"])
user_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 92834 entries, 0 to 92833
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype
---  ------      --------------  -----
 0   user_id     92834 non-null  int64
 1   artist_id   92834 non-null  int64
 2   play_count  92834 non-null  int64
dtypes: int64(3)
memory usage: 2.1 MB


# Peso nelle interazioni

In [ ]:
user_artists["rating"] = np.log1p(user_artists['play_count'])
user_artists[user_artists['user_id'] == 2]

,user_id,artist_id,play_count,rating
0,2,51,13883,9.538492
1,2,52,11690,9.366575
2,2,53,11351,9.337149
3,2,54,10300,9.239996
4,2,55,8983,9.103200
5,2,56,6152,8.724695
6,2,57,5955,8.692154
7,2,58,4616,8.437500
8,2,59,4337,8.375169
9,2,60,4147,8.330382


# Filtraggio di utenti con poche interazioni o artisti con pochi ascolti

In [ ]:
user_interactions_num = user_artists['user_id'].value_counts()
artist_plays_num = user_artists['artist_id'].value_counts()

min_interactions = 10
min_plays = 10


user_artists = user_artists[user_artists['user_id'].isin(user_interactions_num[user_interactions_num >= min_interactions].index)]
user_artists = user_artists[user_artists['artist_id'].isin(artist_plays_num[artist_plays_num >= min_plays].index)]

display(user_artists.head())
display(print("\n\n"))
display(user_artists.info())

,user_id,artist_id,play_count,rating
0,2,51,13883,9.538492
1,2,52,11690,9.366575
2,2,53,11351,9.337149
3,2,54,10300,9.239996
4,2,55,8983,9.103200


None

<class 'pandas.core.frame.DataFrame'>
Index: 62412 entries, 0 to 92803
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   user_id     62412 non-null  int64  
 1   artist_id   62412 non-null  int64  
 2   play_count  62412 non-null  int64  
 3   rating      62412 non-null  float64
dtypes: float64(1), int64(3)
memory usage: 2.4 MB


None

# Dizionario indici

In [ ]:
usersID_codes = user_artists["user_id"].astype("category")
artistsID_codes = user_artists["artist_id"].astype("category")

user_artists["users_id_code"] = usersID_codes.cat.codes
user_artists["artists_id_code"] = artistsID_codes.cat.codes

display(user_artists.head())
display(print("\n\n"))
display(user_artists.info())

uid2raw = dict(enumerate(usersID_codes.cat.categories))
aid2raw = dict(enumerate(artistsID_codes.cat.categories))
name_by_raw_artist = dict(zip(artists["artist_id"], artists.get("artist_name", artists["artist_id"])))
name_by_iid = {i: name_by_raw_artist.get(raw_i, f"artist_{raw_i}") for i, raw_i in aid2raw.items()}

,user_id,artist_id,play_count,rating,users_id_code,artists_id_code
0,2,51,13883,9.538492,0,17
1,2,52,11690,9.366575,0,18
2,2,53,11351,9.337149,0,19
3,2,54,10300,9.239996,0,20
4,2,55,8983,9.103200,0,21


None

<class 'pandas.core.frame.DataFrame'>
Index: 62412 entries, 0 to 92803
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   user_id          62412 non-null  int64  
 1   artist_id        62412 non-null  int64  
 2   play_count       62412 non-null  int64  
 3   rating           62412 non-null  float64
 4   users_id_code    62412 non-null  int16  
 5   artists_id_code  62412 non-null  int16  
dtypes: float64(1), int16(2), int64(3)
memory usage: 2.6 MB


None

# Matrice Utente 𝗑 Artista

righe = utenti, colonne = artisti.

In [ ]:
n_users = user_artists["users_id_code"].max()+1
n_items = user_artists["artists_id_code"].max()+1
n_users, n_items

def build_UR(df, n_users, n_items, weight_col="r"):
    return sp.csr_matrix((df[weight_col].values, (df["users_id_code"].values, df["artists_id_code"].values)),
                         shape=(n_users, n_items))

UR_all = build_UR(user_artists, n_users, n_items, "rating")
UR_all

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 62412 stored elements and shape (1799, 1509)>

In [ ]:
UR_df = pd.DataFrame({
    "uid": UR_all.nonzero()[0],   # utenti (righe con valori)
    "iid": UR_all.nonzero()[1],   # item (colonne con valori)
    "weight": UR_all.data          # pesi (log1p o 1)
})

UR_df["user_id"] = UR_df["uid"].map(uid2raw)
UR_df["artist_id"] = UR_df["iid"].map(aid2raw)
UR_df["artist_name"] = UR_df["iid"].map(name_by_iid)

UR_df.to_csv("user_artist_matrix.csv", index=False)

# Split Train/Validation/Test Sets per il Ranking

Per ogni utente viene preso, in modo randomico, 1 item (artista) per il test set, 1 per il validation e il resto è training